In [1]:
using Distributions, Random
using JLD2
using greedy
using Shared
using NumOpt

In [2]:
Random.seed!(113)

TaskLocalRNG()

<h1>Tasks setup</H1>

In [3]:
const T = 96
const Δt = 15.0 / 60
const base_load_total = rand(Normal(1000.0 , 20.0 ), T)
base_load = [base_load_total ./ 3  base_load_total ./ 3  base_load_total ./ 3]
@show size(base_load)
price = rand(Normal(0.8, 0.2), T)
clamp!(price, 0.4, 1.1)
;

size(base_load) = (96, 3)


In [4]:
extrema(base_load_total)

(959.1492756168113, 1052.0311759279086)

In [5]:
P_max = 6000.0
pi_max = 8e-2
;

In [6]:
K = 100
tasks = gen_tasks(T, K)
env = Environment(T, Δt, P_max, pi_max, base_load, price, tasks)
;

<h1> Numerical optimization </h1>

In [7]:
model, solve_time= solve!(env; optimizer=:Cbc, opt_options=Dict("logLevel"=>0, "ratioGap"=>10e-3));
# model, solve_time= solve!(env; optimizer=:Gurobi);

(A JuMP Model
Minimization problem with:
Variables: 9600
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 100 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 672 constraints
`JuMP.VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 3238 constraints
`JuMP.VariableRef`-in-`MathOptInterface.ZeroOne`: 9600 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: P_ch, P_phase, P_phase_avg, P_phase_diff, P_total, cost, found_solution, obj, opt_status, x, 42.144999980926514)

In [8]:
@show solve_time

solve_time = 42.144999980926514


42.144999980926514

In [9]:
# @show model
x = get_decision_matrix(model)
validate_solution_feasibility(x, env)

👍The solution is feasible!


In [10]:
obj = get_value(model[:obj])

3610.4524037302476

In [11]:
get_value(model[:P_total])  |> extrema

(975.161996169734, 1718.6372070665227)

In [12]:
root_dir = @__DIR__ 
jldsave(joinpath(root_dir, "data/tasks113.jld2"); env, x, obj)

<h1> Greedy optimization </h1>

In [13]:
result, GY_solve_time = run_greedy(env)

([1 0 … 1 0; 1 0 … 1 0; … ; 0 0 … 0 0; 0 0 … 0 0], 0.23799991607666016)

In [14]:
get_value_greedy(result, env)

3622.0349754617905

In [15]:
validate_solution_feasibility(result, env)

AssertionError: AssertionError: all(h .== env.tasks.h)